# Sampling-Based Motion Planning: RRT and RRT-Connect

Impressively, we made it through previous chapters without doing any real motion planning! We were able to get away with explicitly defining poses of interest and interpolating between them without any consideration of our environment. Unfortunately, with this approach, navigating cluttered environments while avoiding both collisions and system limitations is quite the challenge. To address this, we'll use motion planning

In this exercise, you'll build your first sampling-based motion planners. We'll start with the famous Rapidly-exploring Random Tree (RRT) algorithm. Next, we will extend it to the popular two-tree variant, RRT-Connect.

**Learning Objectives:**
1. Implement RRT: nearest-neighbor search, steering via intermediate configurations, collision checking along edges, and goal biasing.
2. Implement RRT-Connect: grow two trees (start/goal), alternate extend/connect steps, and splice paths when the trees meet.
3. Compare planners using simple metrics (iterations to solve), and reason about parameter choices (step size, goal bias).

**What you'll build:**
- A collision-free configuration-space path for the IIWA in a Drake simulation.

**Reference (optional):**
- [Kuffner & LaValle, “RRT-Connect: An Efficient Approach to Single-Query Path Planning” (ICRA 2000)](https://www.cs.cmu.edu/afs/cs/academic/class/15494-s12/readings/kuffner_icra2000.pdf).


Let's start by getting our imports set up and launching Meshcat.


In [ ]:
import time
from random import random
from typing import Literal

import numpy as np
from pydrake.all import (
    ConstantVectorSource,
    DiagramBuilder,
    MultibodyPlant,
    Parser,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    Simulator,
    SolutionResult,
    Solve,
    StartMeshcat,
    TrajectorySource,
)
from pydrake.multibody import inverse_kinematics
from pydrake.trajectories import PiecewisePolynomial

from manipulation import running_as_notebook
from manipulation.exercises.trajectories.rrt_planner.robot import (
    ConfigurationSpace,
    Range,
)
from manipulation.exercises.trajectories.rrt_planner.rrt_planning import (
    RRT,
    Problem,
    TreeNode,
)
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.station import LoadScenario, MakeHardwareStation

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

### 2D visualization of the RRT algorithm

Run the cell below to get a glimpse of the RRT algorithm in action

In [ ]:
from IPython.display import Image

Image(
    url="https://upload.wikimedia.org/wikipedia/commons/thumb/6/62/Rapidly-exploring_Random_Tree_%28RRT%29_500x373.gif/450px-Rapidly-exploring_Random_Tree_%28RRT%29_500x373.gif"
)

### Add Scenario
Run the cell below to add the scenario that will be used throughout this notebook.

In [ ]:
scenario_yaml = f"""directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0]
        iiwa_joint_2: [0.5]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.9]
        iiwa_joint_5: [0]
        iiwa_joint_6: [0.65]
        iiwa_joint_7: [1.7]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
    X_PC:
        translation: [-0.25, 0, 0]
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.114]
        rotation: !Rpy {{ deg: [90, 0, 90]}}
- add_model:
    name: table
    file: package://drake_models/manipulation_station/amazon_table_simplified.sdf
- add_weld:
    parent: world
    child: table::amazon_table
    X_PC:
        translation: [0.3257, 0, -0.0127]
- add_model:
    name: cupboard
    file: package://manipulation/hydro/cupboard.sdf
- add_weld:
    parent: world
    child: cupboard::cupboard_body
    X_PC:
        translation: [0.9057, 0, 0.4148]
        rotation: !Rpy {{ deg: [0, 0, 180]}}
- add_model:
    name: camera0
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: world
    child: camera0::base
    X_PC:
        translation: [-0.228895, -0.452176, 0.486308]
        rotation: !Rpy {{ deg: [146.0, 78.0, 170]}}
- add_model:
    name: camera1
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: world
    child: camera1::base
    X_PC:
        translation: [-0.201813, 0.469259, 0.417045]
        rotation: !Rpy {{ deg: [150.0, -76.6, -9.8]}}
- add_model:
    name: camera2
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: world
    child: camera2::base
    X_PC:
        translation: [0.786258, -0.048422, 1.043315]
        rotation: !Rpy {{ deg: [150.0, 1.3, 88]}}
- add_model:
    name: bin
    file: package://manipulation/hydro/bin.sdf
- add_weld:
    parent: world
    child: bin::bin_base
    X_PC:
        translation: [0.2, 0, 0]
        rotation: !Rpy {{deg: [0, 0, 180]}}
- add_model:
    name: mustard
    file: package://manipulation/hydro/006_mustard_bottle.sdf
    default_free_body_pose:
        base_link_mustard:
            base_frame: world
            translation: [0.43, 0, 0.215]
model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {{}}
"""

with open("cupboard_scenario_mustard.yaml", "w") as f:
    f.write(scenario_yaml)

## Provided Utility Classes

Implementing RRT from scratch can be very time-consuming. Below, we have provided you the important features you will need to implement the RRT algorithm. Note that in `RRT_tools`, a robot configuration is referred to as $q$, whereas a node in the RRT tree is referred to as a node. One can access the configuration of a node by 
```
q_sample = node.value
```

In [ ]:
class ManipulationStationSim:
    def __init__(self, is_visualizing: bool = False) -> None:
        builder = DiagramBuilder()
        scenario = LoadScenario(filename="cupboard_scenario_mustard.yaml")
        self.station = builder.AddSystem(
            MakeHardwareStation(scenario, meshcat=meshcat if is_visualizing else None)
        )
        self.plant = self.station.GetSubsystemByName("plant")
        self.scene_graph = self.station.GetSubsystemByName("scene_graph")
        self.is_visualizing = is_visualizing

        # scene graph query output port.
        self.query_output_port = self.scene_graph.GetOutputPort("query")

        self.diagram = builder.Build()

        # contexts
        self.context_diagram = self.diagram.CreateDefaultContext()
        self.context_station = self.diagram.GetSubsystemContext(
            self.station, self.context_diagram
        )
        self.station.GetInputPort("iiwa.position").FixValue(
            self.context_station, np.zeros(7)
        )
        self.station.GetInputPort("wsg.position").FixValue(self.context_station, [0.1])
        self.context_scene_graph = self.station.GetSubsystemContext(
            self.scene_graph, self.context_station
        )
        self.context_plant = self.station.GetMutableSubsystemContext(
            self.plant, self.context_station
        )
        # mark initial configuration
        self.q0 = self.plant.GetPositions(
            self.context_plant, self.plant.GetModelInstanceByName("iiwa")
        )
        if is_visualizing:
            self.DrawStation(self.q0, 0.1, -np.pi / 2, np.pi / 2)

    def SetStationConfiguration(
        self,
        q_iiwa: np.ndarray,
        gripper_setpoint: float,
        left_door_angle: float,
        right_door_angle: float,
    ) -> None:
        """
        :param q_iiwa: (7,) numpy array, joint angle of robots in radian.
        :param gripper_setpoint: float, gripper opening distance in meters.
        :param left_door_angle: float, left door hinge angle, in [0, pi/2].
        :param right_door_angle: float, right door hinge angle, in [0, pi/2].
        :return:
        """
        self.plant.SetPositions(
            self.context_plant,
            self.plant.GetModelInstanceByName("iiwa"),
            q_iiwa,
        )
        self.plant.SetPositions(
            self.context_plant,
            self.plant.GetModelInstanceByName("wsg"),
            [-gripper_setpoint / 2, gripper_setpoint / 2],
        )

        # cabinet doors
        if left_door_angle > 0:
            left_door_angle *= -1
        left_hinge_joint = self.plant.GetJointByName("left_door_hinge")
        left_hinge_joint.set_angle(context=self.context_plant, angle=left_door_angle)

        right_hinge_joint = self.plant.GetJointByName("right_door_hinge")
        right_hinge_joint.set_angle(context=self.context_plant, angle=right_door_angle)

    def DrawStation(
        self,
        q_iiwa: np.ndarray,
        gripper_setpoint: float,
        q_door_left: float,
        q_door_right: float,
    ) -> None:
        if not self.is_visualizing:
            print("collision checker is not initialized with visualization.")
            return
        self.SetStationConfiguration(
            q_iiwa, gripper_setpoint, q_door_left, q_door_right
        )
        self.diagram.ForcedPublish(self.context_diagram)

    def ExistsCollision(
        self,
        q_iiwa: np.ndarray,
        gripper_setpoint: float,
        q_door_left: float,
        q_door_right: float,
    ) -> bool:
        self.SetStationConfiguration(
            q_iiwa, gripper_setpoint, q_door_left, q_door_right
        )
        query_object = self.query_output_port.Eval(self.context_scene_graph)
        collision_pairs = query_object.ComputePointPairPenetration()

        return len(collision_pairs) > 0


class IiwaProblem(Problem):
    def __init__(
        self,
        q_start: np.ndarray,
        q_goal: np.ndarray,
        gripper_setpoint: float,
        left_door_angle: float,
        right_door_angle: float,
        is_visualizing=False,
    ) -> None:
        self.gripper_setpoint = gripper_setpoint
        self.left_door_angle = left_door_angle
        self.right_door_angle = right_door_angle
        self.is_visualizing = is_visualizing

        self.collision_checker = ManipulationStationSim(is_visualizing=is_visualizing)

        # Construct configuration space for IIWA.
        plant = self.collision_checker.plant
        nq = 7
        joint_limits = np.zeros((nq, 2))
        for i in range(nq):
            joint = plant.GetJointByName("iiwa_joint_%i" % (i + 1))
            joint_limits[i, 0] = joint.position_lower_limits().item()
            joint_limits[i, 1] = joint.position_upper_limits().item()

        range_list = []
        for joint_limit in joint_limits:
            range_list.append(Range(joint_limit[0], joint_limit[1]))

        def l2_distance(q: tuple):
            sum = 0
            for q_i in q:
                sum += q_i**2
            return np.sqrt(sum)

        max_steps = nq * [np.pi / 180]  # two degrees
        cspace_iiwa = ConfigurationSpace(range_list, l2_distance, max_steps)

        # Call base class constructor.
        Problem.__init__(
            self,
            x=10,  # not used.
            y=10,  # not used.
            robot=None,  # not used.
            obstacles=None,  # not used.
            start=tuple(q_start),
            goal=tuple(q_goal),
            cspace=cspace_iiwa,
        )

    def collide(self, configuration: np.ndarray) -> bool:
        q = np.array(configuration)
        return self.collision_checker.ExistsCollision(
            q,
            self.gripper_setpoint,
            self.left_door_angle,
            self.right_door_angle,
        )

    def visualize_path(self, path: list[tuple]) -> None:
        if path is not None:
            # show path in meshcat
            for q in path:
                q = np.array(q)
                self.collision_checker.DrawStation(
                    q,
                    self.gripper_setpoint,
                    self.left_door_angle,
                    self.right_door_angle,
                )
                if running_as_notebook:
                    time.sleep(0.2)


class IKSolver:
    def __init__(self) -> None:
        ## setup controller plant
        plant_iiwa = MultibodyPlant(0.0)
        iiwa_file = "package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf"
        iiwa = Parser(plant_iiwa).AddModelsFromUrl(iiwa_file)[0]
        # Define frames
        world_frame = plant_iiwa.world_frame()
        L0 = plant_iiwa.GetFrameByName("iiwa_link_0")
        l7_frame = plant_iiwa.GetFrameByName("iiwa_link_7")
        X_WL0 = RigidTransform([-0.25, 0, 0])
        plant_iiwa.WeldFrames(world_frame, L0, X_WL0)
        plant_iiwa.Finalize()
        plant_context = plant_iiwa.CreateDefaultContext()

        # gripper in link 7 frame
        X_L7G = RigidTransform(
            rpy=RollPitchYaw([np.pi / 2, 0, np.pi / 2]), p=[0, 0, 0.114]
        )
        world_frame = plant_iiwa.world_frame()

        self.world_frame = world_frame
        self.l7_frame = l7_frame
        self.plant_iiwa = plant_iiwa
        self.plant_context = plant_context
        self.X_L7G = X_L7G

    def solve(
        self,
        X_WT: RigidTransform,
        q_guess: np.ndarray | None = None,
        theta_bound: float = 0.01,
        position_bound: float = 0.01,
    ) -> tuple[np.ndarray, bool]:
        """
        plant: a mini plant only consists of iiwa arm with no gripper attached
        X_WT: transform of target frame in world frame
        q_guess: a guess on the joint state sol
        """
        plant = self.plant_iiwa
        l7_frame = self.l7_frame
        X_L7G = self.X_L7G
        world_frame = self.world_frame

        R_WT = X_WT.rotation()
        p_WT = X_WT.translation()

        if q_guess is None:
            q_guess = np.zeros(7)

        ik_instance = inverse_kinematics.InverseKinematics(plant)
        # align frame A to frame B
        ik_instance.AddOrientationConstraint(
            frameAbar=l7_frame,
            R_AbarA=X_L7G.rotation(),
            #   R_AbarA=RotationMatrix(), # for link 7
            frameBbar=world_frame,
            R_BbarB=R_WT,
            theta_bound=position_bound,
        )
        # align point Q in frame B to the bounding box in frame A
        ik_instance.AddPositionConstraint(
            frameB=l7_frame,
            p_BQ=X_L7G.translation(),
            # p_BQ=[0,0,0], # for link 7
            frameA=world_frame,
            p_AQ_lower=p_WT - position_bound,
            p_AQ_upper=p_WT + position_bound,
        )
        prog = ik_instance.prog()
        prog.SetInitialGuess(ik_instance.q(), q_guess)
        result = Solve(prog)
        if result.get_solution_result() != SolutionResult.kSolutionFound:
            return result.GetSolution(ik_instance.q()), False
        return result.GetSolution(ik_instance.q()), True

In [ ]:
class RRT_tools:
    def __init__(self, problem: IiwaProblem) -> None:
        # rrt is a tree
        self.rrt_tree = RRT(TreeNode(problem.start), problem.cspace)
        problem.rrts = [self.rrt_tree]
        self.problem = problem

    def find_nearest_node_in_RRT_graph(self, q_sample: tuple) -> TreeNode:
        nearest_node = self.rrt_tree.nearest(q_sample)
        return nearest_node

    def sample_node_in_configuration_space(self) -> tuple:
        q_sample = self.problem.cspace.sample()
        return q_sample

    def calc_intermediate_qs_wo_collision(
        self, q_start: tuple, q_end: tuple
    ) -> list[tuple]:
        """create more samples by linear interpolation from q_start
        to q_end. Return all samples that are not in collision

        Example interpolated path:
        q_start, qa, qb, (Obstacle), qc , q_end
        returns >>> q_start, qa, qb
        """
        return self.problem.safe_path(q_start, q_end)

    def grow_rrt_tree(self, parent_node: TreeNode, q_sample: tuple) -> TreeNode:
        """
        add q_sample to the rrt tree as a child of the parent node
        returns the rrt tree node generated from q_sample
        """
        child_node = self.rrt_tree.add_configuration(parent_node, q_sample)
        return child_node

    def node_reaches_goal(self, node: TreeNode, tol: float = 1e-2) -> bool:
        "returns true if the node is within tol of goal, false otherwise"
        return self.problem.cspace.distance(node.value, self.problem.goal) <= tol

    def backup_path_from_node(self, node: TreeNode) -> list[tuple]:
        path = [node.value]
        while node.parent is not None:
            node = node.parent
            path.append(node.value)
        path.reverse()
        return path

## Getting Started

Now that we've set up the tools we'll need to implement RRT on our iiwa, let's first generate a problem instance. Let's use the default initial joint state as our starting configuration $q_{start}$. Let's use a pre-defined frame in 3D world as our goal pose. The frame of the goal pose can be viewed in the meshcat visualizer below. 

In [ ]:
env = ManipulationStationSim(True)
q_start = env.q0
R_WG = RotationMatrix(np.array([[0, 1, 0], [1, 0, 0], [0, 0, -1]]).T)
T_WG_goal = RigidTransform(p=np.array([4.69565839e-01, 2.95894043e-16, 0.65]), R=R_WG)
AddMeshcatTriad(meshcat, "goal pose", X_PT=T_WG_goal, opacity=0.5)

The joint states of the goal pose can be computed via inverse kinematics.

In [ ]:
ik_solver = IKSolver()
q_goal, optimal = ik_solver.solve(T_WG_goal, q_guess=q_start)

Given the start and goal states, we now have sufficient information to formulate the pathfinding problem. We use `IiwaProblem` class to store all relevant information about the pathfinding problem. For this exercise, you don't have to know the details of this class.

In [ ]:
gripper_setpoint = 0.1
door_angle = np.pi / 2 - 0.001
left_door_angle = -np.pi / 2
right_door_angle = np.pi / 2

iiwa_problem = IiwaProblem(
    q_start=q_start,
    q_goal=q_goal,
    gripper_setpoint=gripper_setpoint,
    left_door_angle=left_door_angle,
    right_door_angle=right_door_angle,
    is_visualizing=True,
)

# RRT Algorithm

RRT grows a tree rooted at the starting configuration by using random samples drawn from the configuration space. At each step, a sample is drawn and a connection is made between the sample and its nearest neighbor in the tree. In the standard version of RRT, the tree extends by a fixed step in the direction of the sample if it is feasable (passes entirely through free space and obeys all constraints). Otherwise, nothing is added. Some variants, however, use different extension strategies. One such "greedy" variant tries to directly connect the random sample to the closest node in the tree. If the full edge is feasible, the entire edge is added to the tree. Otherwise, the longest collision-free portion of the connecting edge is added to the tree. For this problem, you are free to use either method.

With uniform sampling of the search space, the probability of expanding an existing state is proportional to the size of its Voronoi region. As the largest Voronoi regions belong to the states on the frontier of the search, this means that the tree preferentially expands towards large unsearched areas.

However, it may be useful sometimes to bias our exploration towards the goal. In that case, one can artificially set a probability to use the goal as the next sample. 

The pseudocode of the RRT algorithm is shown below.

  **Algorithm RRT**
    
      Input: q_start, q_goal, max_iterations, prob_sample_goal
      Output: path

      TOOLS ← RRT TOOLS

      for k = 1 to max_interation:
        q_sample ← Generate Random Configuration
        random number ← random()
        if random_number < prob_sample_goal:
            q_sample ← q_goal
        n_near ← Find the nearest node in the tree(q_sample)
        (q_1, q_2, ... q_N) ← Find intermediate q's from n_near to q_sample
        
        // iteratively add the new nodes to the tree to form a new edge
        // for a faster runtime, it can be convenient to only add one or a few of the intermediate q's (i.e. first or last)
        last_node ← n_near
        last_node ← Grow RRT tree from last_node to some/all of intermediate q's
        
        if last node reaches the goal:
            path ← backup the path recursively
            return path
        
      return None

## Implement RRT

**YOUR TASK:** Implement the RRT algorithm below. You may find it significantly easier to use the `RRT_tools`. 
**Note:** In your implementation, you should plan in configuration space. Your implementation will be graded on whether the last node of the path has reached the goal and the path is collision-free.

In [ ]:
def rrt_planning(
    problem: IiwaProblem, max_iterations: int = 1000, prob_sample_q_goal: float = 0.05
) -> tuple[list[tuple] | None, int]:
    """
    Input:
        problem (IiwaProblem): instance of a utility class
        max_iterations: the maximum number of samples to be collected
        prob_sample_q_goal: the probability of sampling q_goal

    Output:
    (path, iterations) (tuple):
        path (list): [q_start, ..., q_goal]. Each element q is a configuration (not an RRT node).
        iterations (int): The number of iterations executed to obtain the solution.
                          If no solution is found, return (None, max_iterations).

    """
    rrt_tools = RRT_tools(problem)
    q_goal = problem.goal
    q_start = problem.start

    return None, max_iterations

**Note:** You may need to run this a few times to find a valid path. Feel free to experiment with the numbers for`max_iterations` and `prob_sample_q_goal`. For `max_iterations = 1000`, it can take ~1 minute to run depending on your implementation.

In [ ]:
path, num_iter = rrt_planning(
    iiwa_problem, max_iterations=1000, prob_sample_q_goal=0.15
)
print(f"Number Iter: {num_iter}")
if path is not None:
    print("Found a path!")
else:
    print("No path found")

## Check your Implementation
Run the autograder below to check your implementation. It make take ~30 seconds to run.

In [ ]:
from manipulation.exercises.grader import Grader
from manipulation.exercises.trajectories.test_rrt_planning import TestRRT

Grader.grade_output([TestRRT], [locals()], "results.json")
Grader.print_test_results("results.json")

## Simulate the Path!
Run the code below to visualize the path on the robot arm. Do you notice the "RRT Dance"?

**Note:** Because some trajectories move very close to objects, small execution errors can lead to slight collisions, visible as flashes of a contact force. This is okay. However, there should not be major collisions with very large forces present.

In [ ]:
scenario = LoadScenario(filename="cupboard_scenario_mustard.yaml")
builder = DiagramBuilder()
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder.AddSystem(station)
plant = station.GetSubsystemByName("plant")

if path is None:
    path = [iiwa_problem.start, iiwa_problem.start]

times = [0.05 * i for i in range(len(path))]
Q = np.column_stack(path)
traj = PiecewisePolynomial.FirstOrderHold(times, Q)

iiwa_src = builder.AddSystem(TrajectorySource(traj))
wsg_src = builder.AddSystem(ConstantVectorSource(np.array([0.1])))

builder.Connect(iiwa_src.get_output_port(), station.GetInputPort("iiwa.position"))
builder.Connect(wsg_src.get_output_port(), station.GetInputPort("wsg.position"))

diagram = builder.Build()

diagram_context = diagram.CreateDefaultContext()
station_context = diagram.GetSubsystemContext(station, diagram_context)
station.GetInputPort("wsg.position").FixValue(station_context, [0.1])

simulation = Simulator(diagram)

ctx = simulation.get_mutable_context()
diagram.ForcedPublish(ctx)

meshcat.StartRecording()

if running_as_notebook:
    simulation.set_target_realtime_rate(1.0)

simulation.AdvanceTo(traj.end_time() if running_as_notebook else 0.1)
meshcat.StopRecording()
meshcat.PublishRecording()

**Answer the following question regarding the properties of the RRT algorithm in Gradescope**

Consider the case where we let our RRT algorithm run forever, i..e max_iterations is set to $\infty$. If there is no path to the goal, will RRT warn you? If there is a path to the goal, will RRT eventually find that path? Explain your reasoning for both cases. 


# RRT Connect Algorithm

RRT-Connect builds on the basic RRT algorithm by growing two trees simultaneously: one rooted at the start and the other at the goal. At each iteration, one tree is expanded towards a random sample, and then the other tree attempts to connect directly to the newly added state. This alternating “extend and connect” strategy allows the two trees to rapidly explore the space from both directions, often meeting in the middle to form a complete path.

Like RRT, the method retains the probabilistic bias towards unexplored regions, but it improves efficiency by aggressively trying to connect the trees whenever possible. In practice, this often yields faster convergence and shorter paths than a single-tree RRT.

The pseudocode of the RRT-Connect algorithm is shown below. For more details, see the [original paper](https://www.cs.cmu.edu/afs/cs/academic/class/15494-s12/readings/kuffner_icra2000.pdf).

 **Algorithm RRT-Connect**

    Input: q_start, q_goal, max_iterations, eps_connect
    Output: path

    // Initialize
    TOOLS      ← RRT-CONNECT TOOLS
    T_start    ← Make a tree at the starting configuration
    T_goal     ← Make a tree at the goal configuration

    //  Main loop
    for it = 1 … MAX_ITERS:
        q_rand  ← Sample Configuration

        T_active, T_other  ← alternate which tree is active each iteration 

        status_a, node_a ← Attempt a single step extension of the active tree towards sample

        if we are not trapped:
            q_new ← the value of node_a // add the new node

            status_b, node_b ← attempt a greedy connection between the other tree and q_new

            if the trees connect:
                // Backtrack partial paths from the connecting point to each root 
                path_a ← backtrack active tree from node_a
                path_b ← backtrack other tree from node_b

                If the active tree is the goal tree:
                    swap(path_a, path_b)

                PATH ← concatenate path_a with a reversed path_b
                return (PATH, it)

## RRT-Connect Utility Class

Below, we have provided you the important additional features you will need to implement the RRT-Connect algorithm. Note that `RRT_Connect_tools` is a subclass of `RRT_tools`. As a reminder, a robot configuration is referred to as $q$, whereas a node in the RRT tree is referred to as a node. One can access the configuration of a node by 
```
q_sample = node.value
```

In [ ]:
class RRT_Connect_tools(RRT_tools):
    def create_new_tree(self, q_root: tuple[float]) -> RRT:
        return RRT(TreeNode(q_root), self.problem.cspace)

    def extend_once(
        self, tree: RRT, q_target: tuple[float], eps_connect: float = 1e-3
    ) -> tuple[Literal["Trapped", "Reached", "Advanced"], TreeNode]:
        "extends tree by one step towards q_target"
        q_near_node = tree.nearest(q_target)
        edge = self.problem.safe_path(q_near_node.value, q_target)
        if len(edge) <= 1:
            return "Trapped", q_near_node

        q_step = edge[1]
        new_node = tree.add_configuration(q_near_node, q_step)

        reached = q_step == q_target
        if not reached:
            if self.problem.cspace.distance(q_step, q_target) <= eps_connect:
                tail_edge = self.problem.safe_path(q_step, q_target)
                if len(tail_edge) > 1 and tail_edge[-1] == q_target:
                    new_node = tree.add_configuration(new_node, q_target)
                    reached = True

        return ("Reached" if reached else "Advanced"), new_node

    def connect_greedy(
        self, tree: RRT, q_target: tuple[float], eps_connect: float = 1e-3
    ) -> tuple[Literal["Trapped", "Reached", "Advanced"], TreeNode | None]:
        status, last = "Advanced", None
        while status == "Advanced":
            status, last = self.extend_once(tree, q_target, eps_connect)
            if status == "Trapped":
                return "Trapped", last
        return "Reached", last

    @staticmethod
    def concat_paths(path_a: list[tuple], path_b: list[tuple]) -> list[tuple]:
        if path_a and path_b and path_a[-1] == path_b[0]:
            return path_a + path_b[1:]
        return path_a + path_b

## Implement RRT-Connect

**YOUR TASK:** Implement the RRT-Connect algorithm below. You may find it significantly easier to use the `RRT_Connect_tools`.

**Note:** In your implementation, you should plan in configuration space. Your implementation will be graded on whether the last node of the path has reached the goal and the path is collision-free.

In [ ]:
def rrt_connect_planning(
    problem: IiwaProblem, max_iterations: int = 1000, eps_connect: float = 1e-2
) -> tuple[list[tuple] | None, int]:
    """
    Input:
        problem (IiwaProblem): instance of a utility class
        max_iterations: the maximum number of samples to be collected
        eps_connect: how close the trees need to be within to connect

    Output:
    (path, iterations) (tuple):
        path (list): [q_start, ..., q_goal]. Each element q is a configuration (not an RRT node).
        iterations (int): The number of iterations executed to obtain the solution.
                          If no solution is found, return (None, max_iterations).

    """

    tools = RRT_Connect_tools(problem)
    T_start = tools.rrt_tree
    T_goal = tools.create_new_tree(problem.goal)

    # TODO: Insert your code

    return None, max_iterations

**Note:** You may need to run this a few times to find a valid path. Feel free to experiment with the numbers for`max_iterations`. For `max_iterations = 1000`, it can take ~1 minute to run depending on your implementation.

In [ ]:
path_connect, num_iter = rrt_connect_planning(iiwa_problem, 1000, 1e-2)
print(f"Number Iter: {num_iter}")
if path is not None:
    print("Found a path!")
else:
    print("No path found")

## Check your Implementation
Run the autograder below to check your implementation

In [ ]:
from manipulation.exercises.grader import Grader
from manipulation.exercises.trajectories.test_rrt_planning import TestRRT_Connect

Grader.grade_output([TestRRT_Connect], [locals()], "results.json")
Grader.print_test_results("results.json")

## Simulate the Path!
Run the code below to visualize the path on the robot arm.

In [ ]:
scenario = LoadScenario(filename="cupboard_scenario_mustard.yaml")
builder = DiagramBuilder()
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder.AddSystem(station)
plant = station.GetSubsystemByName("plant")

if path_connect is None:
    path_connect = [iiwa_problem.start, iiwa_problem.start]

times = [0.05 * i for i in range(len(path_connect))]
Q = np.column_stack(path_connect)
traj = PiecewisePolynomial.FirstOrderHold(times, Q)

iiwa_src = builder.AddSystem(TrajectorySource(traj))
wsg_src = builder.AddSystem(ConstantVectorSource(np.array([0.1])))

builder.Connect(iiwa_src.get_output_port(), station.GetInputPort("iiwa.position"))
builder.Connect(wsg_src.get_output_port(), station.GetInputPort("wsg.position"))

diagram = builder.Build()

diagram_context = diagram.CreateDefaultContext()
station_context = diagram.GetSubsystemContext(station, diagram_context)
station.GetInputPort("wsg.position").FixValue(station_context, [0.1])

simulation = Simulator(diagram)

ctx = simulation.get_mutable_context()
diagram.ForcedPublish(ctx)

meshcat.StartRecording()

if running_as_notebook:
    simulation.set_target_realtime_rate(1.0)

simulation.AdvanceTo(traj.end_time() if running_as_notebook else 0.1)
meshcat.StopRecording()
meshcat.PublishRecording()

# Gradescope Verification
Take a screen recording of the robot following your RRT-Connect trajectory and upload it to gradescope **as an mp4**. The file should be (much) smaller than 500MB. The robot should follow the path from $q_{start}$ to $q_{goal}$ planned by RRT-Connect while avoiding collisions with any obstacles in the environment.
**Note:** Because some trajectories move very close to objects, small execution errors can lead to slight collisions, visible as flashes of a contact force. This is okay. However, there should not be major collisions with very large forces present.